# Домашнее задание № 10. Генерация текста

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов, теорий заговоров, постов в помоечных телеграм каналах, текстов журналистов и СМИ с выразительным стилем). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [1]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 30.3 MB/s 
     |████████████████████████████████| 6.6 MB 55.2 MB/s 
     |████████████████████████████████| 596 kB 61.2 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

In [3]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

Выбираю песни Меладзе ♥

In [9]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
site = 'https://textypesen.com'

r = requests.get('https://textypesen.com/valerii-meladze/')
soup = BeautifulSoup(r.text, 'html.parser')
links = []
for val in soup.body.find_all('div')[2].find_all('div', "artist-box__name-holder"):
    link  = val.a['href']
    links.append(link)
songlist = []
with open("songlist.txt", "w+") as f:
  for i in links:
      link = site+i
      r = requests.get(link)
      soup = BeautifulSoup(r.text, 'html.parser')
      song = soup.find_all('p','font-size-20')[0].get_text().replace('\xa0', ' ')
      data = f.read()
      f.write(str(song))
      songlist.append(song)

In [7]:
import json
path = json.dumps(songlist, ensure_ascii = False)
path

'["Залетела к нам голубка\\nВ незакрытое окно,\\nНо всего одну минутку\\nПобыла со мной.\\nЯ ее манил напрасно,\\nСпелых зерен подсыпал,\\nВидно кто-то в небе ясном\\nГолубицу ждал.\\nВидно кто-то в небе ясном\\nГолубицу поджидал.\\nПапироса, папиро…\\nПогасла уж давно,\\nА зима не за горой…\\nИ дикая черная ночь,\\nНо зима не за горой…\\nГлазами кошки смотрит в окно.\\nПапироса, папиро…\\nПрекрасное вино\\nПотерял я номерок…\\nХотя мне уже все равно\\nСамый счастливый номерок…\\nЛегко запомнить: два-ноль-ноль-ноль\\nтекст песни взят с Залетела к нам голубка\\nВ незакрытое окно,\\nНо всего одну минутку побыла со мной\\nЗалетела на минутку\\nИ исчезла за стеной.\\nПапироса, папиро…\\nПогасла уж давно,\\nА зима не за горой…\\nИ дикая черная ночь,\\nНо зима не за горой…\\nГлазами кошки смотрит в окно.\\nПапироса, папиро…\\nПрекрасное вино\\nПотерял я номерок…\\nХотя мне уже все равно\\nСамый счастливый номерок…\\nЛегко запомнить: два-ноль-ноль-ноль\\nПапироса, папиро…\\nПогасла уж давно,\

In [10]:
from transformers import TextDataset, DataCollatorForLanguageModeling



train_dataset = TextDataset( tokenizer=tokenizer,file_path="songlist.txt", block_size=64, overwrite_cache=True)
  

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [12]:
trainer.train()


***** Running training *****
  Num examples = 324
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 100


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=2.0332955932617187, metrics={'train_runtime': 524.5268, 'train_samples_per_second': 61.77, 'train_steps_per_second': 0.191, 'total_flos': 1058232729600000.0, 'train_loss': 2.0332955932617187, 'epoch': 100.0})

In [13]:
text = "Танго и се ля ви "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=50,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Танго и се ля ви )))))))))))))))))))))))))))
А, что мы на балконе танцуем? )))) Это моя работа: сегодня танцуйте меня, завтра — прощальный танец
Все не по календарю: каждый поворот назад на несколько минут.
Траву и смерть я прошу у луны за синий бархат
Когда у меня все идет так, как тебе хочется.
Мы с тобой, танцуя, не расстаемся на минуткуТолько луна может сделать так быстро.
Мы в душе своей все еще ее эхо разносимВ этот вечер в тишине летней
Видно в небе огонек от свечи.Дни летели в одну секунду.
И было мне странно, что из-за дождяНе виден свет.И была у моря лодка, куда я попал из моря,
Но была в ней большая вода,
Хоть мы никогда ее больше не увидим.
Днем на рассвете туман плыл в лодке моей далекой,
А я на берегу ее сидел и


In [14]:
text = "Ее рука была как три руки"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=50,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Ее рука была как три руки
Словно белые перышки
В последний раз…Но ведь так все и будет
Слова эти словно крылья птахи
Красавицы не бывают плохами
Дай мне другой
Оранжевый, зеленый или голубой
Не стоит печали мне знать
Как жаль иногда мы теряемся
В этом огромном мире
И не узнать что за чудо ты несёшь
А мы говорим как ты
Оранжевая ты наша песня
Дай мне другой, белый или синий
Не стоит печали мне знатьЯ один всегда рядом с тобой я не уйду,
И не уйдешь никогдаТак бывает
Между нами всё та же любовь
Была она раньше холодной как лед
Но однажды ветер стих и пришла тьма
И тогда словно две руки
Словно белые перышки
В последний раз…Но ведь так все и будет,
Слова эти словно крылья птехи
Красавицы не бывают плохимиНет у жизни смысла,
Придут в первый


### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

1) Статьи:
* 1 Radford, A., Narasimhan, K., Salimans, T., & Sutskever, I. (2018). Improving language understanding by generative pre-training.
* 2  Radford, A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2019). Language models are unsupervised multitask learners.
* 3 Brown, T., Mann, B., Ryder, N., Subbiah, M., Kaplan, J. D., Dhariwal, P., ... & Amodei, D. (2020). Language models are few-shot learners.


https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf

https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf

https://arxiv.org/pdf/2005.14165.pdf

2) Их корпус состоял из датасетов  WebText и Common Crawl, корпусов книг и статей википедии.
Высокое качество текстов достигалось фильтрацией по схожести с другими крупыми датасетами,  fuzzy deduplication для сохранения достоверной оценки первоначального набора данных, добавлением других известных корпусов и их аугументацией.